- FILTER SEMESTER (FIRST, SECOND, SUMMER, SUMMER2)
- grade_final should only have numerical values such as (1.00 -5.00 excluding 3.50), INC, and DRP.
- create new column, numerical_grade (Ilipat ang grade_reexam to grade_finals if INC). 

COLUMNS TO BE CLEANED:
- grade_reexam
- grade_finals

CLEANED / FILTERED
- semester 
- schoolyear (with _x)
- drop schoolyear without - EXAMPLE ("2004")

EXTRACT DATA

In [37]:
from pyspark.sql import SparkSession

#jdbc_url = "jdbc:postgresql://192.168.20.11:5432/demo_db"
jdbc_url = "jdbc:postgresql://localhost:5432/local_student_grades"
properties = {
    "user": "postgres", 
    "password": "password",  
    "driver": "org.postgresql.Driver",
    "fetchsize": "10000"
}
postgres_driver_path = "C:\postgresql-42.7.5.jar"



def extract(jdbc_url, table_name, properties, postgres_driver_path):
    """ Extract data from PostgreSQL database using Spark."""
    # Initialize Spark session
    spark = SparkSession.builder \
        .appName("Postgres Connection") \
        .config("spark.jars", postgres_driver_path) \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.executor.memoryOverhead", "2g") \
        .config("spark.driver.memoryOverhead", "2g") \
        .getOrCreate()

    # Extracts data from PostgreSQL database
    df = spark.read.jdbc(
        url=jdbc_url,
        table=table_name,
        properties=properties,
    )
    
    return df, spark

In [38]:
# Extract data once
raw_df, spark = extract(jdbc_url, "raw_student_grades", properties, postgres_driver_path)

# Check the number of rows
row_count = raw_df.count()
print(f'Number of rows: {row_count}')

# Print schema
raw_df.printSchema()

Number of rows: 2009740
root
 |-- id: integer (nullable = true)
 |-- schoolyear: string (nullable = true)
 |-- semester: string (nullable = true)
 |-- code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- credits: integer (nullable = true)
 |-- instructor_id: integer (nullable = true)
 |-- instructor: string (nullable = true)
 |-- srcode: integer (nullable = true)
 |-- fullname: string (nullable = true)
 |-- campus: string (nullable = true)
 |-- college: string (nullable = true)
 |-- program: string (nullable = true)
 |-- grade_final: string (nullable = true)
 |-- grade_reexam: string (nullable = true)
 |-- status: string (nullable = true)



In [39]:
raw_df.show()

+-----+----------+--------+----------+--------------------+-------+-------------+--------------+------+------------------+---------+--------------------+--------------------+-----------+------------+------+
|   id|schoolyear|semester|      code|         description|credits|instructor_id|    instructor|srcode|          fullname|   campus|             college|             program|grade_final|grade_reexam|status|
+-----+----------+--------+----------+--------------------+-------+-------------+--------------+------+------------------+---------+--------------------+--------------------+-----------+------------+------+
|29374| 2023-2024|  SUMMER|  PetE 404|Reservoir Petroph...|      4|         3999|LN3999, FN3999|124744|LN124744, FN124744|ALANGILAN|College of Engine...|BS Petroleum Engi...|        INC|        3.00|PASSED|
|29375| 2022-2023|   FIRST|   GEd 101|Understanding the...|      3|         5320|LN5320, FN5320|124744|LN124744, FN124744|ALANGILAN|College of Engine...|BS Petroleum Engi..

In [40]:
total_count = raw_df.count()
print(f"Total instances: {total_count}")

Total instances: 2009740


In [41]:
from pyspark.sql.functions import col, upper

# Uppercase all supposed to be uppercase data just to be sure
df = raw_df.withColumn('grade_final', upper(col('grade_final')))\
                .withColumn('campus', upper(col('campus')))\
                .withColumn('semester', upper(col('semester')))\
                .withColumn('schoolyear', upper(col('schoolyear'))       
)

CHECK FOR TYPO ERROR AND UNWANTED VALUES UNDER semester COLUMN

In [42]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT semester, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         GROUP BY semester""")
show_data.show(22)

+---------------+----------+
|       semester|occurences|
+---------------+----------+
|          THURD|         1|
|          THIRD|        10|
|         SECOND|    770300|
|           1.75|         1|
|        SUMMER2|        43|
|           1.25|         1|
|SECOND SEMESTER|         1|
|       SECOND_X|         4|
|         SUMMER|     56200|
|             --|         1|
|          FIRST|   1183177|
|THIRD TRIMESTER|         1|
+---------------+----------+



FILTER TYPO ERRORS AND INSTANCES FOR SECOND SEMESTER, AND REMOVE ROWS WITH INVALID semester VALUES

In [43]:
from pyspark.sql.functions import when

valid_semesters = ["FIRST", "SECOND", "SUMMER", "SUMMER2"]
df = df.withColumn(
    "semester",
    when(df["semester"].isin("SECOND_X", "SECOND SEMESTER"), "SECOND")
    .when(df["semester"].isin(valid_semesters), df["semester"])
    .otherwise(None)  # Assign None to invalid semesters so they can be filtered
).filter(col("semester").isNotNull())  # Remove rows where semester is None (invalid)

In [44]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT semester, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         GROUP BY semester""")
show_data.show()

+--------+----------+
|semester|occurences|
+--------+----------+
|  SECOND|    770305|
| SUMMER2|        43|
|  SUMMER|     56200|
|   FIRST|   1183177|
+--------+----------+



CHECK FOR SPECIAL CHARACTERS AND CATEGORICAL DATA IN grade_final COLUMN

In [45]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT grade_final, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         GROUP BY grade_final 
                         ORDER BY grade_final DESC""")
show_data.show(22)

+-----------+----------+
|grade_final|occurences|
+-----------+----------+
|     PASSED|       255|
|          P|        27|
|         OG|      1581|
|        INC|     60402|
|          F|         1|
|        DRP|      2907|
|       5.00|     15360|
|       4.00|       142|
|       3.50|         1|
|       3.00|    118143|
|       2.75|     90548|
|       2.50|    177289|
|       2.25|    160248|
|       2.00|    289305|
|       1.75|    259871|
|       1.50|    452501|
|       1.25|    317688|
|       1.00|     63424|
|         --|         2|
|          -|        19|
|        -- |        11|
+-----------+----------+



REMOVE WHITE SPACES FROM ALL THE COLUMNS

In [46]:
from pyspark.sql.functions import trim

df = df.withColumn('schoolyear', trim('schoolyear'))\
.withColumn('semester', trim('semester'))\
.withColumn('code', trim('code'))\
.withColumn('description', trim('description'))\
.withColumn('credits', trim('credits'))\
.withColumn('instructor_id', trim('instructor_id'))\
.withColumn('instructor', trim('instructor'))\
.withColumn('srcode', trim('srcode'))\
.withColumn('fullname', trim('fullname'))\
.withColumn('campus', trim('campus'))\
.withColumn('college', trim('college'))\
.withColumn('program', trim('program'))\
.withColumn('grade_final', trim('grade_final'))\
.withColumn('grade_reexam', trim('grade_reexam'))\
.withColumn('status', trim('status'))



In [47]:
df.createOrReplaceTempView('sample_data_ai_with_id')

show_data = spark.sql("SELECT * FROM sample_data_ai_with_id")
show_data.show(22)

+-----+----------+--------+------------+--------------------+-------+-------------+--------------+------+------------------+------+--------------------+--------------+-----------+------------+------+
|   id|schoolyear|semester|        code|         description|credits|instructor_id|    instructor|srcode|          fullname|campus|             college|       program|grade_final|grade_reexam|status|
+-----+----------+--------+------------+--------------------+-------+-------------+--------------+------+------------------+------+--------------------+--------------+-----------+------------+------+
|38905| 2023-2024|  SECOND|      CA 101|Institutional Cor...|      3|         4019|LN4019, FN4019|135361|LN135361, FN135361|    PB|College of Arts a...|BS Criminology|       2.50|           -|PASSED|
|38906| 2023-2024|  SECOND|    CRIM 104|Human Behavior an...|      3|         4019|LN4019, FN4019|135361|LN135361, FN135361|    PB|College of Arts a...|BS Criminology|       1.75|           -|PASSED|


In [48]:
df.createOrReplaceTempView('sample_data_ai_with_id')

show_data = spark.sql("""SELECT schoolyear, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         GROUP BY schoolyear 
                         ORDER BY schoolyear DESC""")

show_data.show(50)

+-----------+----------+
| schoolyear|occurences|
+-----------+----------+
|  2024-2025|    392358|
|2023-2024_X|         4|
|  2023-2024|    698500|
|  2022-2023|    551385|
|  2021-2022|    306076|
|  2020-2021|     45721|
|  2019-2022|         1|
|  2019-2020|     10394|
|  2018-2019|      2550|
|  2017-2018|       759|
|  2016-2017|       606|
|  2015-2016|       681|
|  2014-2015|       266|
|  2013-2014|       103|
|  2012-2013|        99|
|  2011-2012|        56|
|  2010-2011|        36|
|  2009-2010|        21|
|  2008-2009|        12|
|  2007-2008|        17|
|  2006-2007|         2|
|  2005-2006|         3|
|  2004-2005|        23|
|       2004|         1|
|  2003-2004|        23|
|  2002-2003|        12|
|  2001-2002|         4|
|  2000-2001|         8|
|  1999-2000|         4|
+-----------+----------+



FILTERING schoolyear AND REMOVE/REPLACE UNWANTED VALUES

In [49]:
from pyspark.sql.functions import regexp_extract, col

# 1. Extract the YYYY-YYYY pattern (handles cases with trailing _X)
df = df.withColumn(
    "schoolyear", regexp_extract(col("schoolyear"), r"(\d{4}-\d{4})", 1)
)

# 2. Filter out rows WITHOUT the YYYY-YYYY pattern
schoolyear_regex = r"^\d{4}-\d{4}$"  # Regex for exact YYYY-YYYY format
df = df.filter(col("schoolyear").rlike(schoolyear_regex))  # Keep only matching rows

In [50]:
df.createOrReplaceTempView('sample_data_ai_with_id')

show_data = spark.sql("""SELECT schoolyear, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         GROUP BY schoolyear 
                         ORDER BY schoolyear DESC""")

show_data.show(50)

+----------+----------+
|schoolyear|occurences|
+----------+----------+
| 2024-2025|    392358|
| 2023-2024|    698504|
| 2022-2023|    551385|
| 2021-2022|    306076|
| 2020-2021|     45721|
| 2019-2022|         1|
| 2019-2020|     10394|
| 2018-2019|      2550|
| 2017-2018|       759|
| 2016-2017|       606|
| 2015-2016|       681|
| 2014-2015|       266|
| 2013-2014|       103|
| 2012-2013|        99|
| 2011-2012|        56|
| 2010-2011|        36|
| 2009-2010|        21|
| 2008-2009|        12|
| 2007-2008|        17|
| 2006-2007|         2|
| 2005-2006|         3|
| 2004-2005|        23|
| 2003-2004|        23|
| 2002-2003|        12|
| 2001-2002|         4|
| 2000-2001|         8|
| 1999-2000|         4|
+----------+----------+



In [51]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT grade_final, grade_reexam, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         GROUP BY grade_final, grade_reexam 
                         ORDER BY occurences DESC""")
show_data.show(80)

+-----------+------------+----------+
|grade_final|grade_reexam|occurences|
+-----------+------------+----------+
|       1.50|           -|    452457|
|       1.25|           -|    317624|
|       2.00|           -|    289284|
|       1.75|           -|    259850|
|       2.50|           -|    177260|
|       2.25|           -|    160236|
|       3.00|           -|    118116|
|       2.75|           -|     90523|
|       1.00|           -|     63412|
|        INC|        3.00|     35140|
|        INC|        5.00|     16175|
|       5.00|           -|     15352|
|        DRP|           -|      2902|
|        INC|           -|      1867|
|         OG|           -|      1581|
|        INC|        2.50|      1469|
|        INC|        2.75|      1437|
|        INC|        2.00|      1294|
|        INC|        2.25|       904|
|        INC|        1.75|       893|
|        INC|        1.50|       847|
|        INC|        1.25|       310|
|     PASSED|           -|       255|
|       4.00

In [52]:
total_count = df.count()
print(f"Total instances: {total_count}")

Total instances: 2009724


CREATING grade_numeric and grade_classification COLUMNS and inserting values accordingly

In [53]:
from pyspark.sql.functions import when, col, regexp_extract, lit, coalesce, isnull, format_number

# 1. Filter out rows with invalid grade_final values (excluding "-", "--")
invalid_grades = ["PASSED", "P", "OG", "F"]
df = df.filter(~col("grade_final").isin(invalid_grades))

# 2. Create the 'grade_numeric' column
df = df.withColumn(
    "grade_numeric",
    when(
        (df["grade_reexam"].isNotNull()) & (~df["grade_reexam"].isin(invalid_grades)) & (df["grade_reexam"].rlike(r"^\d+\.?\d*$")),
        regexp_extract(df["grade_reexam"], r"(\d+\.?\d*)", 1)
    ).when(
        (~df["grade_final"].isin("DRP", "INC")) & (df["grade_final"].rlike(r"^\d+\.?\d*$")),
        regexp_extract(df["grade_final"], r"(\d+\.?\d*)", 1)
    ).when(
       (df["grade_final"] == "INC") & (df["grade_reexam"].isNotNull()) & (~df["grade_reexam"].isin(invalid_grades)) & (df["grade_reexam"].rlike(r"^\d+\.?\d*$")),
        regexp_extract(df["grade_reexam"], r"(\d+\.?\d*)", 1)
    ).when(df["grade_final"] == "DRP", 0
    )
    .otherwise(None)
).withColumn("grade_numeric", col("grade_numeric").cast("double")).fillna({"grade_numeric": 5})


# 3. Filter valid numeric grades
valid_numeric_grades = ['1.00','1.25','1.50','1.75','2.00','2.25','2.50','2.75','3.00','4.00','5.00']
df = df.filter(
    when(~df["grade_numeric"].isin(0.0), df["grade_numeric"].isin([float(x) for x in valid_numeric_grades])).otherwise(True)
)


# Format grade_numeric to two decimal places x.xx
df = df.withColumn("grade_numeric", format_number("grade_numeric", 2))

# 4. Create the 'grade_classification' column
df = df.withColumn(
    "grade_classification",
    when(
        (df["grade_final"].rlike(r"^\d+\.\d+$")) & (isnull(df["grade_reexam"]) | df["grade_reexam"].isin(invalid_grades)),
        'NORMAL')
    .when(
        (df["grade_final"].isin(["-", "--"])) & (df["grade_reexam"].rlike(r"^\d+\.\d+$")),
        'NORMAL')
    .when(
        (df["grade_final"] == "INC") & (df["grade_reexam"].rlike(r"^\d+\.\d+$")),
        'INC')
    .when(
        df["grade_final"] == "DRP",
        'DROP')
    .when(
        ((df["grade_final"].rlike(r"^\d+\.\d+$")) & (df["grade_reexam"] == "INC")) |
        ((df["grade_final"].rlike(r"^\d+\.\d+$")) & (df["grade_reexam"].isin(invalid_grades))),
        'INC')
    .when(
        (df["grade_final"].rlike(r"^\d+\.\d+$")) & (~df["grade_reexam"].rlike(r"^\d+\.\d+$")),
        'NORMAL')
    .when(
        (df["grade_final"].rlike(r"^\d+\.\d+$")) & (df["grade_reexam"].rlike(r"^\d+\.\d+$")) &
        (col("grade_final").cast("double") > col("grade_reexam").cast("double")),
        'IMPROVED(REEXAM)')
    .when(
        (df["grade_final"].rlike(r"^\d+\.\d+$")) & (df["grade_reexam"].rlike(r"^\d+\.\d+$")) &
        (col("grade_final").cast("double") < col("grade_reexam").cast("double")),
        'FAILED(REEXAM)')
            
    .otherwise("INVALID")
)


In [54]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- schoolyear: string (nullable = true)
 |-- semester: string (nullable = true)
 |-- code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- credits: string (nullable = true)
 |-- instructor_id: string (nullable = true)
 |-- instructor: string (nullable = true)
 |-- srcode: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- campus: string (nullable = true)
 |-- college: string (nullable = true)
 |-- program: string (nullable = true)
 |-- grade_final: string (nullable = true)
 |-- grade_reexam: string (nullable = true)
 |-- status: string (nullable = true)
 |-- grade_numeric: string (nullable = true)
 |-- grade_classification: string (nullable = false)



In [55]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT grade_final, grade_reexam, grade_numeric, grade_classification, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         GROUP BY grade_final, grade_reexam, grade_numeric, grade_classification
                         ORDER BY occurences DESC""")
show_data.show(80)

+-----------+------------+-------------+--------------------+----------+
|grade_final|grade_reexam|grade_numeric|grade_classification|occurences|
+-----------+------------+-------------+--------------------+----------+
|       1.50|           -|         1.50|              NORMAL|    452457|
|       1.25|           -|         1.25|              NORMAL|    317624|
|       2.00|           -|         2.00|              NORMAL|    289284|
|       1.75|           -|         1.75|              NORMAL|    259850|
|       2.50|           -|         2.50|              NORMAL|    177260|
|       2.25|           -|         2.25|              NORMAL|    160236|
|       3.00|           -|         3.00|              NORMAL|    118116|
|       2.75|           -|         2.75|              NORMAL|     90523|
|       1.00|           -|         1.00|              NORMAL|     63412|
|        INC|        3.00|         3.00|                 INC|     35140|
|        INC|        5.00|         5.00|           

In [56]:
total_count = df.count()
print(f"Total instances: {total_count}")

Total instances: 2007859


FILTER OUT (OG, P, Passed, F, DRP)

In [57]:
# Filter out the data that is not included on the list
allowed_data = ['1.00','1.25','1.50',
                '1.75','2.00','2.25',
                '2.50','2.75','3.00',
                '4.00','5.00', 'INC', 'DRP']

In [58]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT program, count(*)
                         FROM sample_data_ai_with_id
                         WHERE program = 'NULL'
                         GROUP BY program """)
show_data.show(50)

+-------+--------+
|program|count(1)|
+-------+--------+
|   NULL|    8180|
+-------+--------+



DROP programs with "NULL" strings

In [59]:
df = df.filter(col('program') != 'NULL')

CHECK FOR NULL STRINGS IN program COLUMN

In [60]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT program, count(*)
                         FROM sample_data_ai_with_id
                         WHERE program = 'NULL'
                         GROUP BY program """)
show_data.show(50)

+-------+--------+
|program|count(1)|
+-------+--------+
+-------+--------+



In [61]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT grade_final, grade_reexam, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         WHERE grade_reexam = '-' OR grade_reexam = '--' OR grade_reexam = 'NULL' OR grade_reexam IS NULL
                         GROUP BY grade_final, grade_reexam 
                         ORDER BY occurences DESC""")
show_data.show(50)

+-----------+------------+----------+
|grade_final|grade_reexam|occurences|
+-----------+------------+----------+
|       1.50|           -|    450611|
|       1.25|           -|    316203|
|       2.00|           -|    288146|
|       1.75|           -|    258147|
|       2.50|           -|    176843|
|       2.25|           -|    159565|
|       3.00|           -|    118021|
|       2.75|           -|     90344|
|       1.00|           -|     62813|
|       5.00|           -|     15352|
|        DRP|           -|      2900|
|        INC|           -|      1866|
|       1.25|          --|        34|
|       1.50|          --|        26|
|       2.50|        null|        22|
|       3.00|        null|        18|
|       2.75|        null|        14|
|       2.00|        null|        11|
|          -|           -|        10|
|       1.00|          --|         9|
|       2.25|        null|         9|
|       3.00|          --|         8|
|        INC|          --|         8|
|       2.75

ONLY ALLOW NUMERICAL STRINGS IN GRADE_REEXAM COLUMN

In [62]:
df = df.withColumn(
    "grade_reexam", when(col("grade_reexam").rlike("^[0-9]*\.?[0-9]+$"), col("grade_reexam")).otherwise(None)
)

In [63]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT grade_final, grade_reexam, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         WHERE grade_final = 'INC' OR grade_final = 'DRP' 
                         GROUP BY grade_final, grade_reexam 
                         ORDER BY occurences DESC""")
show_data.show(22)

+-----------+------------+----------+
|grade_final|grade_reexam|occurences|
+-----------+------------+----------+
|        INC|        3.00|     35134|
|        INC|        5.00|     16174|
|        DRP|        null|      2903|
|        INC|        null|      1877|
|        INC|        2.50|      1467|
|        INC|        2.75|      1433|
|        INC|        2.00|      1292|
|        INC|        2.25|       904|
|        INC|        1.75|       892|
|        INC|        1.50|       845|
|        INC|        1.25|       309|
|        INC|        1.00|        50|
+-----------+------------+----------+



In [64]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT grade_final, grade_reexam, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id
                         GROUP BY grade_final, grade_reexam 
                         ORDER BY occurences DESC""")
show_data.show(22)

+-----------+------------+----------+
|grade_final|grade_reexam|occurences|
+-----------+------------+----------+
|       1.50|        null|    450644|
|       1.25|        null|    316238|
|       2.00|        null|    288162|
|       1.75|        null|    258158|
|       2.50|        null|    176869|
|       2.25|        null|    159576|
|       3.00|        null|    118047|
|       2.75|        null|     90365|
|       1.00|        null|     62822|
|        INC|        3.00|     35134|
|        INC|        5.00|     16174|
|       5.00|        null|     15360|
|        DRP|        null|      2903|
|        INC|        null|      1877|
|        INC|        2.50|      1467|
|        INC|        2.75|      1433|
|        INC|        2.00|      1292|
|        INC|        2.25|       904|
|        INC|        1.75|       892|
|        INC|        1.50|       845|
|        INC|        1.25|       309|
|       4.00|        3.00|       116|
+-----------+------------+----------+
only showing

In [65]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT schoolyear, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         WHERE schoolyear = '2023-2024_x' 
                         GROUP BY schoolyear
                         ORDER BY occurences DESC""")
show_data.show(22)

+----------+----------+
|schoolyear|occurences|
+----------+----------+
+----------+----------+



In [66]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT campus, COUNT(*) AS occurences 
                         FROM sample_data_ai_with_id 
                         GROUP BY campus
                         ORDER BY occurences DESC""")

show_data.show()

+---------+----------+
|   campus|occurences|
+---------+----------+
|ALANGILAN|    745300|
|       PB|    530632|
|  NASUGBU|    239742|
|   MALVAR|    189020|
|     LIPA|    155623|
|  ROSARIO|     41678|
|   LEMERY|     36747|
|  BALAYAN|     27566|
| SAN JUAN|     16036|
|     LOBO|     12597|
|   MABINI|      4735|
|     null|         3|
+---------+----------+



In [67]:
from pyspark.sql.types import DecimalType

filtered_df = df.withColumn("credits", df.credits.cast("int"))\
    .withColumn("grade_numeric", df.grade_numeric.cast(DecimalType(5, 2)))


In [68]:
filtered_df.createOrReplaceTempView('data')
show_data = spark.sql("""SELECT * 
                         FROM data
                        WHERE schoolyear = '2024-2025' AND grade_final = 'INC' AND grade_reexam is NULL
                        """)

show_data.show()

+-----+----------+--------+---------+--------------------+-------+-------------+--------------+------+------------------+---------+--------------------+--------------------+-----------+------------+------+-------------+--------------------+
|   id|schoolyear|semester|     code|         description|credits|instructor_id|    instructor|srcode|          fullname|   campus|             college|             program|grade_final|grade_reexam|status|grade_numeric|grade_classification|
+-----+----------+--------+---------+--------------------+-------+-------------+--------------+------+------------------+---------+--------------------+--------------------+-----------+------------+------+-------------+--------------------+
|49812| 2024-2025|   FIRST| ARCH 525|Computer-Aided De...|      2|         4911|LN4911, FN4911|105468|LN105468, FN105468|ALANGILAN|College of Archit...|     BS Architecture|        INC|        null|FAILED|         5.00|             INVALID|
|52506| 2024-2025|   FIRST|  GEd 103

DROP RECORDS WITH 0.00 as grade_numeric AND RECORDS WITH INC-NULL AS grade_numeric WITH THE CURRENT YEAR IN SCHOOLYEAR

In [69]:
from datetime import datetime

current_year = datetime.now().year

# Drop rows where grade_final is INC and reexam is null
filtered_df = filtered_df.filter(
    ~((col('grade_final') == 'INC') & (col('grade_reexam').isNull())) & (col('schoolyear').contains(str(current_year)))
)

filtered_df = filtered_df.filter((col('grade_numeric') != 0))

In [70]:
filtered_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- schoolyear: string (nullable = true)
 |-- semester: string (nullable = true)
 |-- code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- credits: integer (nullable = true)
 |-- instructor_id: string (nullable = true)
 |-- instructor: string (nullable = true)
 |-- srcode: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- campus: string (nullable = true)
 |-- college: string (nullable = true)
 |-- program: string (nullable = true)
 |-- grade_final: string (nullable = true)
 |-- grade_reexam: string (nullable = true)
 |-- status: string (nullable = true)
 |-- grade_numeric: decimal(5,2) (nullable = true)
 |-- grade_classification: string (nullable = false)



In [71]:
filtered_df.show(20)

+-----+----------+--------+------------+--------------------+-------+-------------+--------------+------+------------------+---------+--------------------+--------------------+-----------+------------+------+-------------+--------------------+
|   id|schoolyear|semester|        code|         description|credits|instructor_id|    instructor|srcode|          fullname|   campus|             college|             program|grade_final|grade_reexam|status|grade_numeric|grade_classification|
+-----+----------+--------+------------+--------------------+-------+-------------+--------------+------+------------------+---------+--------------------+--------------------+-----------+------------+------+-------------+--------------------+
|88357| 2024-2025|   FIRST|   AnSCI 409|Ruminant Producti...|      3|         1536|LN1536, FN1536|113680|LN113680, FN113680|     LOBO|College of Agricu...|      BS Agriculture|       1.50|        null|PASSED|         1.50|              NORMAL|
|88358| 2024-2025|   FIR

In [72]:
filtered_df.createOrReplaceTempView('data')
show_data = spark.sql("""SELECT * 
                         FROM data
                        WHERE schoolyear = '2024-2025' AND grade_final = 'INC' AND grade_reexam is NULL
                        """)

show_data.show()

+---+----------+--------+----+-----------+-------+-------------+----------+------+--------+------+-------+-------+-----------+------------+------+-------------+--------------------+
| id|schoolyear|semester|code|description|credits|instructor_id|instructor|srcode|fullname|campus|college|program|grade_final|grade_reexam|status|grade_numeric|grade_classification|
+---+----------+--------+----+-----------+-------+-------------+----------+------+--------+------+-------+-------+-----------+------------+------+-------------+--------------------+
+---+----------+--------+----+-----------+-------+-------------+----------+------+--------+------+-------+-------+-----------+------------+------+-------------+--------------------+



In [1]:
filtered_df.count()

NameError: name 'filtered_df' is not defined

CREATE A TEMPORARY DATABASE THAT WRITES THE FILTERED_DF INTO POSTGRESQL

In [73]:
filtered_df.write \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "filtered_data_with_id") \
        .option("user", properties["user"]) \
        .option("password", properties["password"]) \
        .option("driver", properties["driver"]) \
        .mode("overwrite") \
        .save()

In [70]:
df.createOrReplaceTempView('sample_data_ai_with_id')
show_data = spark.sql("""SELECT grade_numeric
                         FROM sample_data_ai_with_id """)
show_data.show(22)

+-------------+
|grade_numeric|
+-------------+
|         2.00|
|         1.50|
|         1.00|
|         1.25|
|         1.50|
|         1.75|
|         1.50|
|         1.25|
|         1.25|
|         1.25|
|         1.50|
|         1.00|
|         1.25|
|         1.50|
|         1.50|
|         1.25|
|         2.50|
|         2.50|
|         2.75|
|         2.25|
|         2.50|
|         3.00|
+-------------+
only showing top 22 rows



In [71]:
filtered_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- schoolyear: string (nullable = true)
 |-- semester: string (nullable = true)
 |-- code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- credits: integer (nullable = true)
 |-- instructor_id: string (nullable = true)
 |-- instructor: string (nullable = true)
 |-- srcode: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- campus: string (nullable = true)
 |-- college: string (nullable = true)
 |-- program: string (nullable = true)
 |-- grade_final: string (nullable = true)
 |-- grade_reexam: string (nullable = true)
 |-- status: string (nullable = true)
 |-- grade_numeric: decimal(5,2) (nullable = true)
 |-- grade_classification: string (nullable = false)



PSEUDOCODE FOR CREATING TRAINING DFs

In [ ]:

converted_df = df.franz

for program in converted_df.program:
    for srcode in converted_df.srcode:
        for schoolyear_semester in converted_df.schoolyear_semester:
            for avg_type in converted_df.avg_type:
                #compute GWA
                if min:
                    #use min_grades
                    #insert min_avg
                    pass
                elif mid:
                    #use mid_grades
                    #insert mid_avg
                    pass
                else:
                    #use max_grades
                    #insert max_avg
                    pass
                
    df.create()